In [273]:
import pandas as pd
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split 
#import tensorflow as tf

In [364]:
def load_csv(path):     
    csv_path = path
    if csv_path != "":
        df = pd.read_csv(csv_path, index_col=0)
        print(df)
        return df
    
    else:
        return None

In [371]:
DUMP_FILE_PATH = "S:\\HIT-SOAR\\SOAR\\Uploads\\exported.csv"
df = load_csv(DUMP_FILE_PATH)

              createdat                                            logtext
id                                                                        
11  2023-01-31 17:03:22  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
12  2023-01-31 17:03:22  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
13  2023-01-31 17:03:22  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
14  2023-01-31 17:03:23  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
15  2023-01-31 17:03:24  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
16  2023-01-31 17:03:24  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
17  2023-01-31 17:03:24  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
18  2023-01-31 17:03:25  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
19  2023-01-31 17:03:25  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...
20  2023-01-31 17:03:25  {\r\n  "Event": {\r\n    "#attributes": {\r\n ...


In [562]:
def list_to_json(source):
    source = source.replace("\t", '" ,"')
    source = source.replace("\r", "")
    source = source.replace("\n", "")
    source = source.replace("=", '":"')
    source = source.replace("\'", "")
    source = source.replace('\\', '/')
    source = "{ " + source[3:] + ' " }'
    return source

In [563]:
def extract_features(source_column):
    res = pd.DataFrame()
    rec_list = source_column.to_list()
    for record in rec_list:
        temprec = pd.json_normalize(json.loads(record)) 
        res = pd.concat([res, temprec], axis='index', ignore_index=True)
     
    return res

In [565]:
dfx = extract_features(df[df.columns[1]])
dfx

,Event.#attributes.xmlns,Event.EventData.Binary,Event.EventData.Data.#text,Event.System.Channel,Event.System.Computer,Event.System.Correlation,Event.System.EventID.#attributes.Qualifiers,Event.System.EventID.#text,Event.System.EventRecordID,Event.System.Execution.#attributes.ProcessID,Event.System.Execution.#attributes.ThreadID,Event.System.Keywords,Event.System.Level,Event.System.Opcode,Event.System.Provider.#attributes.Name,Event.System.Security,Event.System.Task,Event.System.TimeCreated.#attributes.SystemTime,Event.System.Version
0,http://schemas.microsoft.com/win/2004/08/event...,None,"[FileSystem, Started, \tProviderName=FileSyste...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,600,22,10428,0,0x80000000000000,4,0,PowerShell,None,6,2023-01-31T17:03:22.998210Z,0
1,http://schemas.microsoft.com/win/2004/08/event...,None,"[Function, Started, \tProviderName=Function\r\...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,600,23,10428,0,0x80000000000000,4,0,PowerShell,None,6,2023-01-31T17:03:22.998636Z,0
2,http://schemas.microsoft.com/win/2004/08/event...,None,"[Variable, Started, \tProviderName=Variable\r\...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,600,24,10428,0,0x80000000000000,4,0,PowerShell,None,6,2023-01-31T17:03:22.999107Z,0
3,http://schemas.microsoft.com/win/2004/08/event...,None,"[Available, None, \tNewEngineState=Available\r...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,400,25,10428,0,0x80000000000000,4,0,PowerShell,None,4,2023-01-31T17:03:23.052788Z,0
4,http://schemas.microsoft.com/win/2004/08/event...,None,[ Add-Type -AssemblyName System.IO.Comp...,Windows PowerShell,DESKTOP-Q3IFKSG,None,0,800,26,10428,0,0x80000000000000,4,0,PowerShell,None,8,2023-01-31T17:03:24.203163Z,0
5,http://schemas.microsoft.com/win/2004/08/event...,None,[ Add-Type -AssemblyName System.IO.Comp...,Windows PowerShell,DESKTOP-Q3IFKSG,None,0,800,27,10428,0,0x80000000000000,4,0,PowerShell,None,8,2023-01-31T17:03:24.205710Z,0
6,http://schemas.microsoft.com/win/2004/08/event...,None,"[Stopped, Available, \tNewEngineState=Stopped\...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,403,28,10428,0,0x80000000000000,4,0,PowerShell,None,4,2023-01-31T17:03:24.301038Z,0
7,http://schemas.microsoft.com/win/2004/08/event...,None,"[Registry, Started, \tProviderName=Registry\r\...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,600,29,10804,0,0x80000000000000,4,0,PowerShell,None,6,2023-01-31T17:03:25.241765Z,0
8,http://schemas.microsoft.com/win/2004/08/event...,None,"[Alias, Started, \tProviderName=Alias\r\n\tNew...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,600,30,10804,0,0x80000000000000,4,0,PowerShell,None,6,2023-01-31T17:03:25.241958Z,0
9,http://schemas.microsoft.com/win/2004/08/event...,None,"[Environment, Started, \tProviderName=Environm...",Windows PowerShell,DESKTOP-Q3IFKSG,None,0,600,31,10804,0,0x80000000000000,4,0,PowerShell,None,6,2023-01-31T17:03:25.242039Z,0


Adding random Y column to build pipeline

In [383]:
import random
randomlist = []
for i in range(0,len(df)):
    n = random.randint(0,1)
    randomlist.append(n)

dfx['results'] = pd.DataFrame(randomlist)

In [384]:
X = pd.get_dummies(dfx.drop(['results', "Event.EventData.Data.#text"], axis=1))
y = dfx['results']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

Build model from here

In [385]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim= len(X_train.columns)),
    tf.keras.layers.Dense(1,  activation='sigmoid')
])

In [386]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

In [387]:
model.fit(X_train, y_train, epochs= 200, batch_size=32)

Epoch 1/200
1/1 [==============================] - 0s 180ms/step - loss: 34.5436 - accuracy: 0.7143
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 2616498.2500 - accuracy: 0.2857
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 0.6922 - accuracy: 0.7143
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 0.6918 - accuracy: 0.7143
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 0.6913 - accuracy: 0.7143
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 0.6909 - accuracy: 0.7143
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 0.6904 - accuracy: 0.7143
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.7143
Epoch 9/200
1/1 [==============================] - 0s 3ms/step - loss: 0.6895 - accuracy: 0.7143
Epoch 10/200
1/1 [==============================] - 0s 3ms/step - loss: 0.6891 - accuracy: 0.7143
Epoch 11/200
1/1 [==

In [388]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

1/1 [==============================] - 0s 23ms/step


In [390]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat)

0.6666666666666666

Finally saving the model using: model.save("SOARmodel")